<a href="https://colab.research.google.com/github/yuyehara/IS-GEO2022_Angeve/blob/main/clippingRaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopandas
!pip install earthpy
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.3 MB/s 
     |████████████████████████████████| 6.3 MB 44.1 MB/s 
     |████████████████████████████████| 16.7 MB 38.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 4.5 MB/s 
     |████████████████████████████████| 19.3 MB 389 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Original code ref.: https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/crop-raster-data-with-shapefile-in-python/
# import libraries
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from shapely.geometry import mapping
import rasterio
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep

In [3]:
# Prettier plotting with seaborn
sns.set(font_scale=1.5)

In [4]:
# get vector using geopanda
palm_0339_poly = gpd.read_file("/content/drive/MyDrive/IS-GEO-22/palm_0339.shp")

# get a raster
palm_0339_ras = rasterio.open("/content/drive/MyDrive/IS-GEO-22/img_0339.tif")

In [5]:
# check the coodinate reference system (crs)
polyCrs = palm_0339_poly.crs
rasCrs = palm_0339_ras.crs

print("palm_0339_poly =", polyCrs)
print("palm_0339_ras =", rasCrs)

palm_0339_poly = epsg:4326
palm_0339_ras = EPSG:32604
